In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [3]:
summary_and_visit_df= pd.read_csv("../data/summary_and_visit_df")

In [5]:
sorted_sum= summary_and_visit_df.sort_values(by=["patient_id", "visit_date"])

In [6]:
sorted_sum.drop(columns=["Unnamed: 0"],inplace=True)

In [7]:
sorted_sum.reset_index(inplace=True,drop=True)

In [8]:
sorted_sum.dtypes

patient_id     int64
age            int64
visit_date    object
summary       object
num_visits     int64
dtype: object

In [9]:
sorted_sum["visit_date"]=pd.to_datetime(sorted_sum["visit_date"])

Get the time in between visits

In [10]:
sorted_sum["time_between_visits"]=""

for i in range(sorted_sum.shape[0]):
    try:
        patient_sum= sorted_sum.loc[sorted_sum['patient_id'] == sorted_sum['patient_id'][i]]
        sorted_sum["time_between_visits"][i]= patient_sum["visit_date"][i+1] - patient_sum["visit_date"][i]
    except: 
        i= i+1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [11]:
sorted_sum.dropna(subset=["time_between_visits"],inplace=True)

Turn datetime object to a float so we can use it for modelling 

In [12]:
def split_dates(dates_obj):
    if dates_obj:
        to_string=str(dates_obj)
        date_num= to_string.split()        
        return float(date_num[0])
    else: 
        return 

In [13]:
sorted_sum["time_between_visits"]= sorted_sum["time_between_visits"].map(split_dates)

Get the average time between visits for each patient

In [14]:
values_time= sorted_sum.groupby(by='patient_id',as_index=False)['time_between_visits'].mean()

Create a column to add average time between visits for each patient

In [15]:
sorted_sum["avg_time_between_visits"]=""

for x in range(sorted_sum.shape[0]):
    for i in range(len(values_time)):
        try:
            if sorted_sum["patient_id"][x]== values_time["patient_id"][i]:
                #index of where patient id is that value 
                sorted_sum["avg_time_between_visits"][x]= values_time["time_between_visits"][i]
        except:
            continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
sorted_sum.to_csv("../data/ss.csv")